In [1]:
import psychrolib as psy
psy.SetUnitSystem(psy.IP)
import pandas as pd
import numpy as np

In [47]:
#file with data
fileName = r"C:\Users\renjesse\University of Toronto\Deepal Patel - Shared Folder - Co-op\Working Folder\Data Analysis\CWEC\2018 Comparison\AC-7 2018 data.csv"

In [35]:
#file being created
newFileName = r"C:\Users\renjesse\University of Toronto\Deepal Patel - Shared Folder - Co-op\Working Folder\Data Analysis\CWEC\2018 Comparison\Calculated AC-17B 2018 data.csv"

In [50]:
#converts csv file to dataframe, if you receive a UnicodeDecodeError concerning the utf-8 codec, open the csv file in 
#notepad, press 'Save As' and then save it with UTF-8 encoding
df = pd.read_csv(fileName)

In [37]:
airFlow = list(df["Total Air Flow Proportional to Power(cfm)"]) #enter name of column with airflow

In [38]:
airTempBeingCooled = list(df[" OutsideAirTemp (°F)"]) #enter name of column with temperature that is being cooled

In [52]:
enteringRH= list(df["MAH"]) #enter name of column with RH of mixed air 

In [39]:
defaultRH = []
for i in range(len(airFlow)): #default RH is 100, assign it to supplyRH is supplyRH is always 100
    defaultRH.append(100) 
supplyRH = defaultRH #if SupplyRH is not always 100, make supplyRH equal to list(df["columnname"]), where column name is the name of the column with supplyRH

In [40]:
supplyTemp = list(df["AC17 SupplyAirTemp (°F)"]) #enter name of column with supply temperature

In [41]:
OAH = list(df["DewPointTemp (°F)"]) #enter name of column with OA RH

In [35]:
RAH = list(df[".RAH"]) #enter name of column with RA RH

In [16]:
TDewPoint = list(df["Dew Point(?F)"]) #enter name of column with Dewpoint Temperature

In [42]:
OAT = list(df[" OutsideAirTemp (°F)"]) #enter name of column with OAT

In [38]:
RAT = list(df["RAT (?F) "]) #enter name of column with RAT

In [39]:
percentRA = list(df["RA%"]) #enter name of column with percent of MA that is RA
percentOA = list(df["OA%"]) #enter name of column with percent of MA that is OA

In [17]:
OAEnthalpy = [] #run this line if calculated for OA enthalpy

In [40]:
MAH = [] #run this line if calculated for MA RH

In [43]:
coolingLoad = [] #run this line if calculated for cooling loads

In [58]:
#Get Cooling Load from mixed air and MAH

#runs through each line in file
for i in range(len(airFlow)):
    #checks if RH of current line is valid and if the supplyTemp is cooler than the entering Temp
    if enteringRH[i] != 0 and airTempBeingCooled[i] > supplyTemp[i]: 
        #checks if the enthalpy of the entering air is greater than the enthalpy of the supply air
        if psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(airTempBeingCooled[i],enteringRH[i]/100,14.7))-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)) > 0: 
            coolingLoad.append(4.5*airFlow[i]*(psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(airTempBeingCooled[i],enteringRH[i]/100,14.7))-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)))/1000)
        else:
            #appends only the sensible cooling load if the enthalpy of the entering air is less than the enthalpy of the supply air
            coolingLoad.append(1.1*airFlow[i]*(airTempBeingCooled[i]-supplyTemp[i])/1000)
    else:
        #appends a blank space if either the RH or entering temperature are not valid
        coolingLoad.append("")

In [20]:
#Get Cooling Load from mixed air and OA% and RA%
for i in range(len(airFlow)):
    if OAH[i] != 0 and airTempBeingCooled[i] > supplyTemp[i]:
        if psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100)-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)) > 0:
            coolingLoad.append(4.5*airFlow[i]*(psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100)-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)))/1000)
        else:
            coolingLoad.append(1.1*airFlow[i]*(airTempBeingCooled[i]-supplyTemp[i])/1000)
    else:
        coolingLoad.append("")

In [157]:
#Get Cooling Load from TDewPoint and OA% and RA%
for i in range(len(airFlow)):
    if TDewPoint != "" and airTempBeingCooled[i] > supplyTemp[i]:
        if psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i],14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100)-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)) > 0:
            coolingLoad.append(4.5*airFlow[i]*(psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i],14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100)-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)))/1000)
        else:
            coolingLoad.append(1.1*airFlow[i]*(airTempBeingCooled[i]-supplyTemp[i])/1000)
    else:
        coolingLoad.append("")

In [44]:
#Get Cooling Load from Relative Humidity and Outside Air
for i in range(len(airFlow)):
    if OAH != 0 and airTempBeingCooled[i] > supplyTemp[i]:
        if psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7))-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)) > 0:
            coolingLoad.append(4.5*airFlow[i]*(psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7))-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)))/1000)
        else:
            coolingLoad.append(1.1*airFlow[i]*(airTempBeingCooled[i]-supplyTemp[i])/1000)
    else:
        coolingLoad.append("")

In [230]:
#Get Cooling Load from TDewPoint and Outside Air
for i in range(len(airFlow)):
    if TDewPoint[i] != "" and airTempBeingCooled[i] > supplyTemp[i]:
        if psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i],14.7))-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)) > 0:
            coolingLoad.append(4.5*airFlow[i]*(psy.GetMoistAirEnthalpy(airTempBeingCooled[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i],14.7))-psy.GetMoistAirEnthalpy(supplyTemp[i],psy.GetHumRatioFromRelHum(supplyTemp[i],supplyRH[i]/100,14.7)))/1000)
        else:
            coolingLoad.append(1.1*airFlow[i]*(airTempBeingCooled[i]-supplyTemp[i])/1000)
    else:
        coolingLoad.append("")

In [56]:
#Get MAH from TDewPoint
for i in range(len(airFlow)):
    MAH.append(psy.GetRelHumFromHumRatio(airTempBeingCooled[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i],14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100,14.7)*100)


In [41]:
#Get MAH from Relative Humidity

for i in range(len(airFlow)):
    if OAH[i] != 0:
        if psy.GetRelHumFromHumRatio(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100,14.7)*100 <= 100
            MAH.append(psy.GetRelHumFromHumRatio(airTempBeingCooled[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7)*percentOA[i]/100+psy.GetHumRatioFromRelHum(RAT[i],RAH[i]/100,14.7)*percentRA[i]/100,14.7)*100)
        else:
            Mah.append(100)
    else:
        MAH.append("")

In [19]:
#Get OA Enthalpy from OAT and RH%
for i in range(len(airFlow)):
    if OAH[i] != 0:
        OAEnthalpy.append(psy.GetMoistAirEnthalpy(OAT[i],psy.GetHumRatioFromRelHum(OAT[i],OAH[i]/100,14.7)))
    else:
        OAEnthalpy.append("")

In [ ]:
#Get OA Enthalpy from OAT and TDewPoint%
for i in range(len(airFlow)):
    OAEnthalpy.append(psy.GetMoistAirEnthalpy(OAT[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i]/100,14.7)))


In [45]:
df2 = pd.DataFrame(coolingLoad) #enter list with calculated results in brackets

In [46]:
df2.to_csv(newFileName,sep=",") #run to create csv with calculated results

In [139]:
OAEnthalpy = [] #Get Outdoor Air Enthalpy
for i in range(len(TDewPoint)):
    if OAT[i] != '' and TDewPoint[i] != "": #checks if the current lines OAT and TDewPoint are not blank
        OAEnthalpy.append(psy.GetMoistAirEnthalpy(OAT[i],psy.GetHumRatioFromTDewPoint(TDewPoint[i],14.7)))
    else:
        OAEnthalpy.append("")           
                          
df3 = pd.DataFrame(OAEnthalpy)
df3.to_csv(newFileName,sep=",")

In [205]:
wetBulb = [] #Get Outdoor Wetbulb Temperature
for i in range(len(airFlow)):
    if OAT[i] != '':
        wetBulb.append(psy.GetTWetBulbFromHumRatio(float(airTempBeingCooled[i]),psy.GetHumRatioFromRelHum(float(OAT[i]),float(OAH[i]) / 100,14.7),14.7))
    else:
        wetBulb.append("")
df3 = pd.DataFrame(wetBulb)
df3.to_csv(newFile,sep=",")